# Spam detection using Transformers Models

In this notebook, we will leverage transformers models such as BERT to classify whether an SMS contains spam or not.


### * What is a Transformer model?

A transformer is a deep learning model that adopts the mechanism of self-attention. It is used primarily in the fields of **Natural Language Processing (NLP)** and Computer Vision (CV). 

**BERT** (Bidirectional Encoder Representations from Transformers) is one of the most famous transformer models released by Google in 2018.

### * Which family of Machine Learning paradigms does this task (spam detection) belong to?

The task of detecting if an SMS contains spam or not is a **supervised** task. Each example is associated with a **label** (1: SPAM, 0: not SPAM). Since the labels are discrete, they are classes, the task is a **classification** (binary classification because #classes = 2)

### * Which libraries are we going to use?

We are going to use one of the most famous libraries to work with Transformers model, the name is the library is (you do not need a lot of fantasy) *transformers* by *HuggingFace*.


# Libraries installation

In [ ]:
!pip install transformers --quiet

In [ ]:
!pip install datasets --quiet

In [ ]:
!pip install pandas -U --quiet

In [ ]:
!pip install --quiet shap

# Dataset

The dataset is hosted in the *Huggingface* dataset hub and can be easly downloaded.

In [ ]:
from datasets import load_dataset
# https://huggingface.co/datasets/sms_spam
spam_dataset = load_dataset("sms_spam", split = ["train"])

The dataset is composed by 5574 examples and has 2 columns:
- *sms*: the text of the sms.
- *label*: 1 if spam, 0 otherwise.

In [ ]:
spam_dataset[0]

Let's have a look at some rows...

In [ ]:
spam_dataset[0][100]

In [ ]:
spam_dataset[0][2]

How are the labels distributed? Let's see if the dataset is unbalanced.

In [ ]:
from collections import Counter
Counter([sample['label'] for sample in spam_dataset[0]])

# Model

The model is hosted in the *Huggingface* model hub and can be easily downloaded.

Someone (thanks) already trained (better to say finetuned) a BERT model using the dataset we have seen above. Thus we are not focusing on the training part, but remember that training a neural network requires time and resources (GPU/TPU). One of the pros of using the transformers library is that: 
* Researchers can share trained models instead
* Practitioners can reduce compute time and production costs


## Bert 

Bidirectional Encoder Representations from Transformers (BERT) is a transformer-based machine learning technique for natural language processing (NLP) pre-training developed by Google. BERT was created and published in 2018.

**Architecture**

The original English-language BERT has been released into 2 versions:
- BERT BASE: 12 encoders with 12 bidirectional - self-attention heads. (110 M parameters)
- BERT LARGE: 24 encoders with 16 bidirectional - self-attention heads. (345 M parameters)

**Data**

Both models are pre-trained from unlabeled data extracted from the BooksCorpus with 800M words and English Wikipedia with 2,500M words.

**Tasks**

BERT was pre-trained on two tasks (self-supervised): 

- Masked Language Modelling: 15% of tokens were masked and BERT was trained to predict them from context. (You can try it [here](https://huggingface.co/bert-base-uncased) to better understand this task).
- Next Sentence Prediction: BERT was trained to predict if a chosen next sentence was probable or not given the first sentence. 

**Result**

After pretraining, which is computationally expensive, BERT can be finetuned with fewer resources on smaller datasets to optimize its performance on specific tasks.


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# the name of a model (that is on the model hub is this case)
model_name = "mariagrandury/distilbert-base-uncased-finetuned-sms-spam-detection" #mariagrandury/distilbert-base-uncased-finetuned-sms-spam-detection

# let's load the model 
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# let's load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Tokenizer

A tokenizer is in charge of preparing the inputs for the model. When the BERT model was trained, each token was given a unique ID. Therefore, when we want to use a pre-trained BERT model, we will first need to convert each token in the input sentence into its corresponding unique IDs.


BERT uses a WordPiece algorithm that breaks a word into several subwords: we cannot say word = token.

BERT has some special token that are: [CLS], [SEP], [PAD] and [UNK].

In [ ]:
sample = 'This is an example to show you how is done the tokenization process.'
encoding = tokenizer.encode(sample)
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding))

# **Testing**

Legend: 

* LABEL_1 -> spam

* LABEL_2 -> not spam

* score -> how much ""confident"" the model is about the predicted label [0;1]

In [ ]:
from transformers import TextClassificationPipeline
 # a pipeline is very easy-to-use abstraction, which require as little as two lines of code to perform a prediction given a model and a tokenizer
classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer)
classifier("Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days.")

Let's try it

In [ ]:
# good prediction :)
classifier("Hey John, do you like Machine Learning?")

In [ ]:
# good prediction :)
classifier("Urgent! you are the selected winner of 1 bitcoin, answer YES to confirm your price.")

In [ ]:
# good prediction :)
classifier("Hey Luis, you call Peter at 093232141? I'm in a meeting right now, see you later.")

In [ ]:
# bad prediction :(
classifier("Answer YES, to get the chance to win a Ferrari, 2$ per month.")

## Metrics

WARNING: The data we are testing likely has been used to train the model (we should use a different split of the dataset that we don't have) but the goal is to show you how to calculate the metric not looking at the specific number

In [ ]:
references = [sample['label'] for sample in spam_dataset[0]][:1000]
input_texts = [sample['sms'] for sample in spam_dataset[0]][:1000]

In [ ]:
predictions = classifier(input_texts)

In [ ]:
label_mapping = {"LABEL_0": 0, "LABEL_1": 1}
predictions = [label_mapping[pred_dict["label"]] for pred_dict in predictions]

Let's review some metrics for classification: 

$Accuracy = \frac{TP+TN}{TP+TN+FP+FN}$

$Precision = \frac{TP}{TP+FP}$

$Recall = \frac{TP}{TP+FN}$

$F1 = \frac{2*Precision*Recall}{Precision+Recall} = \frac{2*TP}{2*TP+FP+FN}$







In [ ]:
from datasets import load_metric

load_f1 = load_metric("accuracy")
accuracy = load_f1.compute(predictions=predictions, references=references)["accuracy"]
print("Accuracy: ", accuracy)

The dataset is quite imbalanced, it is better to look at the F1 score

In [ ]:
load_f1 = load_metric("f1")
f1 = load_f1.compute(predictions=predictions, references=references)["f1"]
print("F1 score: ", f1)

# Explanability


## What if we don't want just a number but a deeper understanding of which part of the text influenced the prediction?

Shap to the rescue

In [ ]:
import shap
explainer = shap.Explainer(classifier)
shap_values = explainer(["Urgent! you are the selected winner of 1 bitcoin, answer YES to confirm your price."])

In [ ]:
shap.plots.text(shap_values[:,:,"LABEL_1"])

In [ ]:
shap.plots.bar(shap_values[0,:,"LABEL_1"])